<a href="https://colab.research.google.com/github/likunglaif/Pembelajaran-Mesin-Course/blob/main/Week%2014/Week_14_MRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --upgrade pip
!pip install tensorflow
!pip install --upgrade keras

     |████████████████████████████████| 1.7 MB 5.0 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


In [3]:
from random import randint
from numpy import array
from numpy import argmax
from pandas import DataFrame
from pandas import concat
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
 
# generate a sequence of random numbers in [0, 99]
def generate_sequence(length=25):
	return [randint(0, 99) for _ in range(length)]
 
# one hot encode sequence
def one_hot_encode(sequence, n_unique=100):
	encoding = list()
	for value in sequence:
		vector = [0 for _ in range(n_unique)]
		vector[value] = 1
		encoding.append(vector)
	return array(encoding)
 
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
	return [argmax(vector) for vector in encoded_seq]
 
# convert encoded sequence to supervised learning
def to_supervised(sequence, n_in, n_out):
	# create lag copies of the sequence
	df = DataFrame(sequence)
	df = concat([df.shift(n_in-i-1) for i in range(n_in)], axis=1)
	# drop rows with missing values
	df.dropna(inplace=True)
	# specify columns for input and output pairs
	values = df.values
	width = sequence.shape[1]
	X = values.reshape(len(values), n_in, width)
	y = values[:, 0:(n_out*width)].reshape(len(values), n_out, width)
	return X, y
 
# prepare data for the LSTM
def get_data(n_in, n_out):
	# generate random sequence
	sequence = generate_sequence()
	# one hot encode
	encoded = one_hot_encode(sequence)
	# convert to X,y pairs
	X,y = to_supervised(encoded, n_in, n_out)
	return X,y
 
# define LSTM
n_in = 5
n_out = 5
encoded_length = 100
batch_size = 7
model = Sequential()
model.add(LSTM(20, batch_input_shape=(batch_size, n_in, encoded_length), return_sequences=True, stateful=True))
model.add(TimeDistributed(Dense(encoded_length, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# train LSTM
for epoch in range(500):
	# generate new random sequence
	X,y = get_data(n_in, n_out)
	# fit model for one epoch on this sequence
	model.fit(X, y, epochs=1, batch_size=batch_size, verbose=2, shuffle=False)
	model.reset_states()
# evaluate LSTM
X,y = get_data(n_in, n_out)
yhat = model.predict(X, batch_size=batch_size, verbose=0)
# decode all pairs
for i in range(len(X)):
	print('Expected:', one_hot_decode(y[i]), 'Predicted', one_hot_decode(yhat[i]))

3/3 - 2s - loss: 4.6113 - accuracy: 0.0000e+00 - 2s/epoch - 669ms/step
3/3 - 0s - loss: 4.6010 - accuracy: 0.0095 - 12ms/epoch - 4ms/step
3/3 - 0s - loss: 4.6011 - accuracy: 0.0190 - 11ms/epoch - 4ms/step
3/3 - 0s - loss: 4.6004 - accuracy: 0.0000e+00 - 12ms/epoch - 4ms/step
3/3 - 0s - loss: 4.6080 - accuracy: 0.0000e+00 - 12ms/epoch - 4ms/step
3/3 - 0s - loss: 4.6054 - accuracy: 0.0095 - 15ms/epoch - 5ms/step
3/3 - 0s - loss: 4.6047 - accuracy: 0.0381 - 11ms/epoch - 4ms/step
3/3 - 0s - loss: 4.6038 - accuracy: 0.0000e+00 - 11ms/epoch - 4ms/step
3/3 - 0s - loss: 4.6043 - accuracy: 0.0190 - 11ms/epoch - 4ms/step
3/3 - 0s - loss: 4.5940 - accuracy: 0.0571 - 12ms/epoch - 4ms/step
3/3 - 0s - loss: 4.5990 - accuracy: 0.0190 - 14ms/epoch - 5ms/step
3/3 - 0s - loss: 4.6052 - accuracy: 0.0095 - 12ms/epoch - 4ms/step
3/3 - 0s - loss: 4.5891 - accuracy: 0.1429 - 11ms/epoch - 4ms/step
3/3 - 0s - loss: 4.5920 - accuracy: 0.0000e+00 - 11ms/epoch - 4ms/step
3/3 - 0s - loss: 4.5923 - accuracy: 0.0571

In [5]:
from random import randint
from numpy import array
from numpy import argmax
from pandas import DataFrame
from pandas import concat
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import RepeatVector
 
# generate a sequence of random numbers in [0, 99]
def generate_sequence(length=25):
	return [randint(0, 99) for _ in range(length)]
 
# one hot encode sequence
def one_hot_encode(sequence, n_unique=100):
	encoding = list()
	for value in sequence:
		vector = [0 for _ in range(n_unique)]
		vector[value] = 1
		encoding.append(vector)
	return array(encoding)
 
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
	return [argmax(vector) for vector in encoded_seq]
 
# convert encoded sequence to supervised learning
def to_supervised(sequence, n_in, n_out):
	# create lag copies of the sequence
	df = DataFrame(sequence)
	df = concat([df.shift(n_in-i-1) for i in range(n_in)], axis=1)
	# drop rows with missing values
	df.dropna(inplace=True)
	# specify columns for input and output pairs
	values = df.values
	width = sequence.shape[1]
	X = values.reshape(len(values), n_in, width)
	y = values[:, 0:(n_out*width)].reshape(len(values), n_out, width)
	return X, y
 
# prepare data for the LSTM
def get_data(n_in, n_out):
	# generate random sequence
	sequence = generate_sequence()
	# one hot encode
	encoded = one_hot_encode(sequence)
	# convert to X,y pairs
	X,y = to_supervised(encoded, n_in, n_out)
	return X,y
 
# define LSTM
n_in = 5
n_out = 2
encoded_length = 100
batch_size = 21
model = Sequential()
model.add(LSTM(150, batch_input_shape=(batch_size, n_in, encoded_length), stateful=True))
model.add(RepeatVector(n_out))
model.add(LSTM(150, return_sequences=True, stateful=True))
model.add(TimeDistributed(Dense(encoded_length, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# train LSTM
for epoch in range(1000):
	# generate new random sequence
	X,y = get_data(n_in, n_out)
	# fit model for one epoch on this sequence
	model.fit(X, y, epochs=1, batch_size=batch_size, verbose=2, shuffle=False)
	model.reset_states()
# evaluate LSTM
X,y = get_data(n_in, n_out)
yhat = model.predict(X, batch_size=batch_size, verbose=0)
# decode all pairs
for i in range(len(X)):
	print('Expected:', one_hot_decode(y[i]), 'Predicted', one_hot_decode(yhat[i]))

1/1 - 3s - loss: 4.6042 - accuracy: 0.0000e+00 - 3s/epoch - 3s/step
1/1 - 0s - loss: 4.6042 - accuracy: 0.0000e+00 - 14ms/epoch - 14ms/step
1/1 - 0s - loss: 4.6073 - accuracy: 0.0714 - 13ms/epoch - 13ms/step
1/1 - 0s - loss: 4.6038 - accuracy: 0.0238 - 14ms/epoch - 14ms/step
1/1 - 0s - loss: 4.6064 - accuracy: 0.0000e+00 - 14ms/epoch - 14ms/step
1/1 - 0s - loss: 4.6039 - accuracy: 0.0000e+00 - 12ms/epoch - 12ms/step
1/1 - 0s - loss: 4.6067 - accuracy: 0.0238 - 16ms/epoch - 16ms/step
1/1 - 0s - loss: 4.6026 - accuracy: 0.0476 - 15ms/epoch - 15ms/step
1/1 - 0s - loss: 4.6059 - accuracy: 0.0238 - 13ms/epoch - 13ms/step
1/1 - 0s - loss: 4.6041 - accuracy: 0.0238 - 20ms/epoch - 20ms/step
1/1 - 0s - loss: 4.6019 - accuracy: 0.0952 - 13ms/epoch - 13ms/step
1/1 - 0s - loss: 4.6028 - accuracy: 0.0476 - 12ms/epoch - 12ms/step
1/1 - 0s - loss: 4.6043 - accuracy: 0.0952 - 13ms/epoch - 13ms/step
1/1 - 0s - loss: 4.5954 - accuracy: 0.1190 - 14ms/epoch - 14ms/step
1/1 - 0s - loss: 4.6035 - accuracy: 